# 📘 Week1: GPT-5 챗봇 실습 (Colab)

### 실습 목표
- Responses API를 사용한 멀티턴 대화 흐름을 이해하고 직접 구현한다.
- 개발자 지침과 사용자 입력을 조합해 챗봇의 성격을 설계한다.
- 위젯 UI를 통해 사용자 경험 측면의 개선 포인트를 탐색한다.

## 1) 환경 준비 및 단순 출력

In [ ]:
pip install openai ipywidgets

### API 키 설정 및 Client 생성
- Colab 메뉴 `보안 비밀(열쇠 아이콘)`에 `OPENAI_API_KEY`를 저장해 두었다면 자동으로 불러옵니다.
- 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")

### OpenAI API Quick Start
- 아래 `create_greeting` 함수를 완성해 API 호출 코드를 직접 작성해 보세요.
- 함수 구현이 완료되면 주석 처리된 예제를 실행해 결과를 검증합니다.

In [ ]:
def create_greeting(client, model, instructions, user_prompt):
    """OpenAI API 호출 실습 함수.

    Args:
        client: OpenAI 클라이언트 인스턴스.
        model: 사용할 모델 ID (예: "gpt-5-nano").
        instructions: 모델에게 전달할 시스템/개발자 지시.
        user_prompt: 사용자 입력 메시지.
    Returns:
        모델이 생성한 최종 텍스트 응답.
    """
    # TODO 1) client.responses.create(...)를 호출할 때 필요한 매개변수를 정리하세요.
    # TODO 2) 모델이 반환한 response 객체에서 최종 텍스트를 꺼내 반환하세요.
    raise NotImplementedError("TODO: Responses API 호출 로직을 구현하세요.")


In [ ]:
# TODO: 위 함수를 구현한 뒤 아래 예제를 실행해 결과를 확인하세요.
greeting = create_greeting(
    client=client,
    model="gpt-5-nano",
    instructions="You are a helpful assistant. Reply in Korean.",
    user_prompt="오늘 하루를 기분 좋게 시작할 수 있는 인사말을 한 문장으로 만들어 주세요."
)
print(greeting)

💡 실습 팁: `model`, `instructions`, 입력 문장을 다양하게 바꿔 보면서 응답의 길이와 어조가 어떻게 달라지는지 비교해 보세요.
예시:
  - **model**: gpt-5, gpt-5-mini, gpt-5-nano 등
  - **instruction1**: "You are a friendly pirate. Always speak like a pirate with 'arr', 'matey', and 'aye'. Reply in Korean but keep the pirate style."
  - **instruction2**: "You are a scholarly professor. Reply in Korean using formal academic language, complex sentences, and technical vocabulary."

## 2) 멀티턴 대화 구성
Responses API는 자동으로 대화 히스토리를 저장하지 않습니다. 따라서 애플리케이션이 직접 이전 메시지를 누적해 전달해야 멀티턴 문맥이 유지됩니다.
- 아래 `request_chat_completion` 함수를 완성해 멀티턴 호출 로직을 직접 작성해 보세요.
- 함수는 전달받은 `history`를 수정하지 않고 Responses API에 전달해야 합니다.

In [ ]:
def request_chat_completion(
    client,
    model,
    history,
    *,
    reasoning_effort="medium",
    verbosity="medium"
):
    """Responses API 멀티턴 대화 실습 함수.

    Args:
        client: OpenAI 클라이언트 인스턴스.
        model: 사용할 모델 ID (예: "gpt-5-mini").
        history: role/content 쌍으로 구성된 대화 히스토리 (함수 내부에서 수정하지 마세요).
        reasoning_effort: 모델의 추론 강도 설정.
        verbosity: 모델의 출력 답변의 길이에 영향을 주는 설정
    Returns:
        (모델이 생성한 텍스트, 원본 response 객체)
    """
    # TODO 1) history를 history_payload 변수에 저장하세요.

    # TODO 2) 각 매개변수가 주어졌을 때 매개변수를 추가하세요.
  
    # TODO 3) response.output_text와 response를 튜플로 반환하세요.
    raise NotImplementedError("TODO: 히스토리를 전달하여 Responses API를 호출하세요.")

In [ ]:
# TODO: 함수 구현 후 아래 셀을 실행하여 멀티턴 흐름을 검증하세요.
conversation = [
    {
        "role": "developer",
        "content": "You are a helpful assistant. Reply in Korean and keep answers concise.",
    },
    {
        "role": "user",
        "content": "방금 읽은 책을 한 줄로 요약해 달라고 부탁하는 방법을 알려 줘.",
    },
]

first_text, first_response = request_chat_completion(
    client=client,
    model="gpt-5-mini",
    history=conversation,
    reasoning_effort="medium",
    verbosity="medium"
)
print("첫 번째 응답:", first_text)

conversation.append({"role": "assistant", "content": first_text})
conversation.append({"role": "user", "content": "이 내용을 더 자세하게 바꿔 줄래?"})

second_text, second_response = request_chat_completion(
    client=client,
    model="gpt-5-mini",
    history=conversation,
    verbosity="high"
)
print("\n두 번째 응답:", second_text)

## 3) 챗봇 만들기 실습
- `ChatSession.send`는 `request_chat_completion` 함수 구현이 완료되어야 정상 동작합니다.
- 필요하다면 `request_chat_completion` 내부를 재사용하여 중복 코드를 줄여 보세요.

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ChatSession:
    """Responses API 기반 멀티턴 챗봇 세션 관리 유틸리티."""

    client: OpenAI
    model: str = "gpt-5-mini"
    developer_message: str = (
        "You are a helpful assistant. You must answer in Korean and keep answers clear."
    )
    reasoning_effort: str = "medium",
    verbosity: str = "medium",
    history: list = field(default_factory=list)
    last_response_id: object = field(default=None, init=False)

    def __post_init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        """대화 히스토리를 초기 developer 메시지로 리셋합니다."""
        # TODO 1) self.history를 developer 메시지 한 건만 담긴 리스트로 초기화하세요.
        # TODO 2) 마지막 응답 ID(self.last_response_id)를 None으로 되돌려 새 세션을 시작하세요.
        raise NotImplementedError("TODO: reset 로직을 직접 구현하세요.")

    def send(self, user_message: str):
        """사용자 메시지를 추가하고 모델 응답 텍스트와 response 객체를 반환합니다."""
        cleaned = user_message.strip()
        if not cleaned:
            raise ValueError("user_message는 비어 있을 수 없습니다.")

        # TODO 1) 히스토리에 사용자 메시지를 추가하고 request_chat_completion을 호출하세요.
        # TODO 2) 모델 응답을 히스토리에 추가하고 last_response_id를 저장한 뒤 (텍스트, response)를 반환하세요.
        raise NotImplementedError("TODO: send 로직을 직접 구현하세요.")

    def pretty_history(self):
        """현재 히스토리를 읽기 쉬운 문자열로 변환합니다."""
        lines = []
        for msg in self.history:
            prefix = "사용자" if msg["role"] == "user" else "assistant" if msg["role"] == "assistant" else msg["role"]
            lines.append(f"[{prefix}] {msg['content']}")
        return "\n".join(lines)


### ChatSession 사용 예시
아래 셀을 실행해 모델과 대화한 뒤 히스토리를 출력해 보세요. `reset()`을 호출하면 처음 상태로 돌아갑니다.


In [ ]:
chatbot = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a helpful teaching assistant. Reply in Korean with actionable suggestions.",
    reasoning_effort="low",
    verbosity = "medium",
)

In [ ]:
reply, raw_response = chatbot.send("OpenAI API 실습을 복습하는 가장 좋은 방법을 알려 줘.")

print("모델 응답:\n", reply)

print("\n현재 히스토리:\n", chatbot.pretty_history())


In [ ]:
raw_response.usage

In [ ]:
# ✏️ 실험해 보기: 아래 developer_message / reasoning_effort / verbosity 등을 바꿔서 다양한 에이전트를 만들어 보세요.
# 예시 1) 해적 말투 상담가 봇
#   developer_message="You are a friendly pirate life coach. Always speak like a pirate mixing a little Korean and say '아르르', '항해', '보물' often. Reply mainly in Korean."
#   예시 입력: "요즘 공부 의욕이 없는데, 해적 선장처럼 동기부여 한 마디만 해줘."
# 예시 2) 여행 코스 추천 봇
#   developer_message="You are an expert Korean travel planner. Suggest fun, practical, and well-organized travel itineraries for Korea or abroad, always in Korean. Include tips and clear day-by-day schedules."
#   예시 입력: "미국 여행가는데 2박 3일 일정 계획 세워줄 수 있어?"
# 예시 3) 냉철한 스타트업 투자자 봇
#   developer_message="You are a ruthless but fair startup investor. Answer only in Korean and critically evaluate the market potential, competition, and business model, honestly pointing out any weaknesses."
#   예시 입력: "고등학생들을 위한 AI 공부 타이머 앱을 만들 건데, 투자할 만한 아이템인지 평가해줘."
# 

> `reasoning.effort` 값을 바꿔 보도록 유도하면 비교 학습이 가능합니다.


## 4) 위젯 기반 챗봇 UI 실습
`ipywidgets`를 활용하면 인터랙티브한 챗봇 실습 환경을 만들 수 있습니다. 아래 UI는 텍스트 입력, 전송 버튼, 초기화 버튼을 제공하며 실행할 때마다 새 세션을 생성합니다.


In [ ]:
import ipywidgets as widgets
from IPython.display import Markdown, display

chat_ui_session = ChatSession(
    client,
    model="gpt-5-mini",
    developer_message="You are a supportive study buddy. Reply in Korean with friendly tips.",
    reasoning_effort="medium",
)

log_output = widgets.Output(layout=widgets.Layout(border="1px solid #ccc", padding="8px"))
input_box = widgets.Text(
    placeholder="질문을 입력하고 Enter 또는 '전송'을 클릭하세요.",
    description="질문",
    layout=widgets.Layout(width="100%"),
)
send_button = widgets.Button(description="전송", button_style="primary")
reset_button = widgets.Button(description="초기화")
status_label = widgets.HTML(value="")


def append_message(role: str, text: str) -> None:
    with log_output:
        display(Markdown(f"**{role}**: {text}"))


def handle_send(_=None) -> None:
    text = input_box.value.strip()
    if not text:
        status_label.value = "<span style='color:#d9534f;'>메시지를 입력하세요.</span>"
        return

    status_label.value = ""
    append_message("사용자", text)
    input_box.value = ""
    send_button.disabled = True
    try:
        answer, _ = chat_ui_session.send(text)
        append_message("어시스턴트", answer)
    except Exception as exc:  # pragma: no cover - 노트북 데모용
        append_message("오류", f"{type(exc).__name__}: {exc}")
    finally:
        send_button.disabled = False


def handle_reset(_=None) -> None:
    chat_ui_session.reset()
    log_output.clear_output()
    status_label.value = "<span style='color:#5cb85c;'>세션이 초기화되었습니다.</span>"


send_button.on_click(handle_send)
reset_button.on_click(handle_reset)
input_box.on_submit(handle_send)

controls = widgets.HBox([send_button, reset_button, status_label])
ui = widgets.VBox([input_box, controls, log_output])

display(ui)

> 실습 지시: 학생들에게 개발자 메시지를 바꿔 보도록 요청해 보세요. 예를 들어 "답변은 표 형식으로 작성"과 같은 추가 지침을 주면 챗봇 동작이 어떻게 달라지는지 확인할 수 있습니다.


### 참고 자료
- @https://platform.openai.com/docs
- @https://platform.openai.com/docs/api-reference
